Imports

In [1]:
import tensorflow as tf
import csv
import numpy as np
from PIL import Image
from matplotlib.pyplot import imshow
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import tensorflow_addons as tfa
import csv

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

'''
from tensorflow.keras.mixed_precision import experimental as mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)
'''

/home/harvey/tf-env/lib/python3.6/site-packages/tensorflow_addons/utils/ensure_tf_install.py:68: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.2.0 and strictly below 2.3.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.3.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


"\nfrom tensorflow.keras.mixed_precision import experimental as mixed_precision\npolicy = mixed_precision.Policy('mixed_float16')\nmixed_precision.set_policy(policy)\n"

Load all the tfrecords.

In [5]:
records = []
from os import walk
for (dirpath, dirnames, filenames) in walk("records/"):
    for f in filenames:
        if "train" in f:
            records.append("records/{}".format(f))

print(records)

['records/train10-2071.tfrec', 'records/train12-2071.tfrec', 'records/train11-2071.tfrec', 'records/train06-2071.tfrec', 'records/train03-2071.tfrec', 'records/train02-2071.tfrec', 'records/train05-2071.tfrec', 'records/train13-2071.tfrec', 'records/train01-2071.tfrec', 'records/train15-2061.tfrec', 'records/train14-2071.tfrec', 'records/train07-2071.tfrec', 'records/train00-2071.tfrec', 'records/train08-2071.tfrec', 'records/train04-2071.tfrec', 'records/train09-2071.tfrec']


In [6]:
dataset = tf.data.TFRecordDataset(filenames = records)

In [7]:
feature_description = {
    'image': tf.io.FixedLenFeature([], tf.string),
    'image_name': tf.io.FixedLenFeature([], tf.string),
    'target': tf.io.FixedLenFeature([], tf.int64),
}

Store the amount of images for each patient in patient_image_count.
<br>
Store a mappting of image name -> patient id in image_to_patient

In [8]:
patient_image_count = {}
image_to_patient = {}

with open("train.csv", "r") as csvfile:
    reader = csv.reader(csvfile, delimiter=",")
    headings = next(reader)
    for im in reader:
        name = im[0]
        patient_id = im[1]
        if patient_id not in patient_image_count:
            patient_image_count[patient_id] = 0
        patient_image_count[patient_id] += 1
        image_to_patient[name] = patient_id
        #print(i, cimages[i]["Target"], ctarget)

In [9]:
IMG_SIZE = 16
channels = 3

#longest = max(patient_image_count.values())
longest = 240
print("Longest sequence is: {}".format(longest))

Longest sequence is: 240


Initialize each patient's structure as follows:<br>
In the 'Targets' array we will store a (longest,) numpy array for each patient containing the targets for each image.<br>
In the 'Masks' array we will store a (longest,) numpy array for each patient containing it's mask.<br>
In the 'Images' array we will store a (longest X IMG_SIZE X IMG_SIZE X 3) numpy array for each patient containing the images.<br>
In the 'Written' array we will store an index of how many examples have been written for each patient, This will be useful in the next step.<br>
In the 'Patient_Order' array we will store a mapping of patient_name -> position in other array, This will be useful in the next step.

In [10]:
targets = []
masks = []
images = []

written = []
patient_order = {}

i = 0

for patient in patient_image_count.keys():
    pad_val = -1
    target_arr = np.full((longest,), pad_val)
    mask_arr = np.ones((longest,))
    image_arr = np.zeros((longest, IMG_SIZE, IMG_SIZE, channels))
    
    targets.append(target_arr)
    masks.append(mask_arr)
    images.append(image_arr)
    
    written.append(0)
    patient_order[patient] = i
    i += 1

Iterate over the dataset and do the following for each example:
<br>
Parse it's image and get the images name.<br>
Find out which patient the image is for.<br>
Find out where in the order this patient is.<br>
Find how many images have been written for this patient.<br>
Increment this patients written value.<br>
Write the image to the correct position in this patients image array.<br>
Write the target to the correct position in this patients target array.<br>
Set the mask value to 0 in the correct position in this patients mask array.

In [11]:
for image in dataset:
    # Parse image.
    example = tf.io.parse_single_example(image, feature_description)
    img = tf.io.decode_image(example['image'], expand_animations = False, channels=channels)
    img = tf.cast(img, tf.float32)
    img = (img/127.5) - 1
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))

    # Get image name.
    name = example['image_name'].numpy().decode("utf_8")

    # Get image target.
    target = example['target'].numpy()
    
    # Work out which patient this belongs to.
    patient = image_to_patient[name]
    
    # Find out patient's index.
    patient_index = patient_order[patient]
    
    # Find how many images have been written to this patient.
    written_index = written[patient_index]
    
    # Write image.
    images[patient_index][written_index, :, :, :] = img
    
    # Write target.
    targets[patient_index][written_index] = target
    
    # Set mask to 0.
    masks[patient_index][written_index] = 0
    
    # Increment written value.
    written[patient_index] += 1

In [12]:
images[0][0].dtype

dtype('float64')

Split into train/val sets

In [13]:
total = len(targets)
#for i in iter(alldata):
#    total += 1
#print("{} images in total.".format(total))

In [14]:
train = 0.66
val = 0.33

train_num = int(total*train)
val_num = int(total*val)

In [15]:
for i in range(len(masks)):
    masks[i] = np.reshape(masks[i], (1, 1, -1))

In [16]:
train_targets = np.array(targets[0:train_num])
val_targets = np.array(targets[train_num:])

train_masks = np.array(masks[0:train_num])
val_masks = np.array(masks[train_num:])

train_images = np.array(images[0:train_num])
val_images = np.array(images[train_num:])

In [17]:
print(train_targets.shape)
print(val_targets.shape)

print(train_images.shape)
print(val_images.shape)

print(train_masks.shape)
print(val_masks.shape)

(1356, 240)
(700, 240)
(1356, 240, 16, 16, 3)
(700, 240, 16, 16, 3)
(1356, 1, 1, 240)
(700, 1, 1, 240)


In [18]:
val_targets

array([[ 0,  0,  0, ..., -1, -1, -1],
       [ 0,  0,  0, ..., -1, -1, -1],
       [ 1,  0,  0, ..., -1, -1, -1],
       ...,
       [ 0,  0,  0, ..., -1, -1, -1],
       [ 0,  0,  1, ..., -1, -1, -1],
       [ 0,  0,  0, ..., -1, -1, -1]])

Define helper functions for model, Most of these are heavily inspired by( Some just copy-pasted ) from the official tensorflow tutorial on Transformer networks.

In [19]:
'''
def scaled_dot_product_attention(q, k, v, mask):
    # Matrix multiply the querys by the keys
    qk = tf.matmul(q, k, transpose_b=True, name="queryXkey")
    
    # Get the dimension of the keys and scale the attention by it
    key_dim = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention = tf.cast(qk, tf.float32)/tf.math.sqrt(key_dim)
    
    # Use the mask to set the masked images( The padding ) to a very small number. Softmax will zero these out.
    canceller = mask*-(10**9)
    
    scaled_attention += canceller
    
    # Softmax the attention weights
    attention_weights = tf.nn.softmax(scaled_attention, axis=-1, name="softmax_attention")
    
    # Multiply the values by the weights
    out = tf.matmul(attention_weights, tf.cast(v, tf.float32), name="attention_out")
    
    return out, attention_weights
    '''
def scaled_dot_product_attention(q, k, v, mask):
  """Calculate the attention weights.
  q, k, v must have matching leading dimensions.
  k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
  The mask has different shapes depending on its type(padding or look ahead) 
  but it must be broadcastable for addition.
  
  Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable 
          to (..., seq_len_q, seq_len_k). Defaults to None.
    
  Returns:
    output, attention_weights
  """

  matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)
  
  # scale matmul_qk
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

  # add the mask to the scaled tensor.
  if mask is not None:
    scaled_attention_logits += (mask * -1e9)  

  # softmax is normalized on the last axis (seq_len_k) so that the scores
  # add up to 1.
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

  output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

  return output, attention_weights

In [20]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model
    
    assert d_model % self.num_heads == 0
    
    self.depth = d_model // self.num_heads
    
    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)
    
    self.dense = tf.keras.layers.Dense(d_model)
        
  def split_heads(self, x, batch_size):
    """Split the last dimension into (num_heads, depth).
    Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    """
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])
    
  def call(self, v, k, q, mask):
    batch_size = tf.shape(q)[0]
    
    q = self.wq(q)  # (batch_size, seq_len, d_model)
    k = self.wk(k)  # (batch_size, seq_len, d_model)
    v = self.wv(v)  # (batch_size, seq_len, d_model)
        
    q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
    k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
    v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)
    
    # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
    # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v, mask)
    
    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(scaled_attention, 
                                  (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

    output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)
        
    return output, attention_weights

In [21]:
def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
  ])

In [22]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(EncoderLayer, self).__init__()

    self.mha = MultiHeadAttention(d_model, num_heads)
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    
    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)
    
  def call(self, x, training, mask):

    attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
    attn_output = self.dropout1(attn_output, training=training)
    out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)
    
    ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
    ffn_output = self.dropout2(ffn_output, training=training)
    out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)
    
    return out2

In [23]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, rate=0.1):
    super(Encoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers
    
    self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) 
                       for _ in range(num_layers)]
          
  def call(self, x, training, mask):
    
    for i in range(self.num_layers):
      x = self.enc_layers[i](x, training, mask)
    
    return x  # (batch_size, input_seq_len, d_model)


In [24]:
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, BatchNormalization

In [25]:
def create_image_embedding_net(filters, out_dim, name):
    inp = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, channels))
    conv = Conv2D(filters, kernel_size=(3, 3), strides=(1, 1), activation="relu")(inp)
    conv = Conv2D(filters//2, kernel_size=(3, 3), strides=(1, 1), activation="relu")(conv)
    maxpool = MaxPool2D(pool_size=(5, 5))(conv)
    flatten = Flatten()(maxpool)
    out = Dense(out_dim)(flatten)
    submodel = tf.keras.Model(name=name, inputs=[inp], outputs=[out])
    return submodel

In [26]:
def create_output_net(vec_shape, dimension, name):
    inp = tf.keras.Input(shape=(vec_shape,))
    internal = Dense(dimension, activation="relu")(inp)
    internal = BatchNormalization()(internal)
    output = Dense(1, activation="sigmoid")(internal)
    submodel = tf.keras.Model(name=name, inputs=[inp], outputs=[output])
    return submodel

In [27]:
embedding_filters = 128

transformer_layers = 6
transformer_dimension = 512
num_heads = 8

transformer_ff_dimension = 2048

out_hidden_dimension = 512

In [28]:
num_images = longest

# Create input for images and masks.
images_in = tf.keras.Input(name="Images_In", shape=(num_images, IMG_SIZE, IMG_SIZE, channels))
mask_in  = tf.keras.Input(name="Mask_In", shape=(1, 1, num_images))

# Create ConvNet to compute query, key and value vectors
embedding_net = create_image_embedding_net(embedding_filters, transformer_dimension, "embedding_net")

embeddings = []

for i in range(num_images):
    img = images_in[:, i, :, :, :]
    embeddings.append(embedding_net(img))

embeddings = tf.stack(embeddings, axis=1)
print("EMBEDDINGS SHAPE:")
print(embeddings.shape)

transformer = Encoder(num_layers=transformer_layers, d_model=transformer_dimension, num_heads=num_heads, dff=transformer_ff_dimension)

new_image_vectors = transformer(embeddings, True, mask_in)
print(new_image_vectors.shape)
print(new_image_vectors[:, 0, :].shape)

output_net = create_output_net(transformer_dimension, out_hidden_dimension, "output_net")

outputs = []

for i in range(num_images):
    img_vec = new_image_vectors[:, i, :]
    cout = output_net(img_vec)
    outputs.append(cout)

print(outputs[0])
output = tf.stack(outputs, axis=-1)[:, 0, :]
print(output.shape)

model = tf.keras.Model(inputs=[images_in, mask_in], outputs=output)

EMBEDDINGS SHAPE:
(None, 240, 512)
(None, 240, 512)
(None, 512)
Tensor("output_net/dense_38/Sigmoid:0", shape=(None, 1), dtype=float32)
(None, 240)


In [ ]:
test_embed_net = create_image_embedding_net(embedding_filters, transformer_dimension, "embedding_net")

In [ ]:
embeddings = []
for i in range(10):
    embeddings.append(test_embed_net(np.array([images[0][i]])))

In [ ]:
embeddings_stack = tf.stack(embeddings, axis=1)

Precision - 
How many of the melanoma that we said are bad were actually bad?
Recall -
How many of the bad melanoma did we catch?

In [ ]:
tf.keras.utils.plot_model(model)

In [ ]:
model.summary()

In [ ]:
'''
def create_loss_func():
    
    bce = tf.keras.losses.BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
    
    def loss_function(real, pred):
        mask = tf.math.logical_not(tf.math.equal(real, -1))
        ctarget = tf.math.equal(real, 1)
        loss_ = bce(ctarget, pred)

        mask = tf.cast(mask, dtype=loss_.dtype)
        print("LMASK")
        print(mask)
        loss_ *= mask
        print("LOSS-MASKED")
        print(loss_)

        return tf.reduce_sum(loss_)/tf.reduce_sum(mask)
    
    return loss_function
'''

In [29]:
def masked_log_loss():
    def loss_function(real, pred):
        epsilon = 9*(10**-5)
        
        mask = tf.math.logical_not(tf.math.equal(real, -1))
        ctarget = tf.math.equal(real, 1)

        #print("PREDICTIONS")
        #print(pred)
        #print("ADDED EPSILON")
        #print(pred+epsilon)
        
        positive_loss = tf.math.log(pred+epsilon)
        negative_loss = tf.math.log((1-pred)+epsilon)
        
        #print("POSITIVE LOSS")
        #print(positive_loss)
        #print("NEGATIVE LOSS")
        #print(negative_loss)
        
        ctarget = tf.cast(ctarget, dtype=positive_loss.dtype)
        
        positive_loss = positive_loss*ctarget
        negative_loss = negative_loss*(1-ctarget)
        
        loss_ = positive_loss+negative_loss
        loss_ *= -1
        
        mask = tf.cast(mask, dtype=loss_.dtype)
        loss_ = loss_*mask
        
        loss_ = tf.reduce_sum(loss_)
        
        return loss_
    return loss_function

In [30]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
          loss=masked_log_loss(),
          metrics=["AUC"])

In [58]:
stopper = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=5,
    verbose=1,
    mode="min",
    restore_best_weights=True
)

#hist = model.fit([train_images, train_masks], train_targets, validation_data=([val_images, val_masks], val_targets),epochs=100, callbacks=[stopper], batch_size=1)

hist = model.fit([train_images, train_masks], train_targets, validation_data=([val_images, val_masks], val_targets), epochs=1000, batch_size=8, callbacks=[stopper])

Epoch 1/1000
170/170 [==============================] - 40s 236ms/step - loss: 12.3781 - auc: 0.6232 - val_loss: 8.5477 - val_auc: 0.6511
Epoch 2/1000
170/170 [==============================] - 41s 239ms/step - loss: 12.3419 - auc: 0.6364 - val_loss: 8.3989 - val_auc: 0.6437
Epoch 3/1000
170/170 [==============================] - 41s 242ms/step - loss: 12.4457 - auc: 0.6291 - val_loss: 8.2332 - val_auc: 0.6527
Epoch 4/1000
 47/170 [=======>......................] - ETA: 25s - loss: 12.9739 - auc: 0.6531

KeyboardInterrupt: 

In [33]:
testrecords = []
from os import walk
for (dirpath, dirnames, filenames) in walk("records/"):
    for f in filenames:
        if "test" in f:
            testrecords.append("records/{}".format(f))

print(testrecords)

['records/test01-687.tfrec', 'records/test02-687.tfrec', 'records/test08-687.tfrec', 'records/test09-687.tfrec', 'records/test10-687.tfrec', 'records/test15-677.tfrec', 'records/test06-687.tfrec', 'records/test04-687.tfrec', 'records/test07-687.tfrec', 'records/test14-687.tfrec', 'records/test05-687.tfrec', 'records/test11-687.tfrec', 'records/test13-687.tfrec', 'records/test00-687.tfrec', 'records/test03-687.tfrec', 'records/test12-687.tfrec']


In [34]:
feature_description_test = {
    'image': tf.io.FixedLenFeature([], tf.string),
    'image_name': tf.io.FixedLenFeature([], tf.string)
}

test_dataset = tf.data.TFRecordDataset(filenames = testrecords)

In [35]:
test_patient_image_count = {}
test_image_to_patient = {}

with open("test.csv", "r") as csvfile:
    reader = csv.reader(csvfile, delimiter=",")
    headings = next(reader)
    for im in reader:
        name = im[0]
        patient_id = im[1]
        if patient_id not in test_patient_image_count:
            test_patient_image_count[patient_id] = 0
        test_patient_image_count[patient_id] += 1
        test_image_to_patient[name] = patient_id
        #print(i, cimages[i]["Target"], ctarget)

In [43]:
test_masks = []
test_images = []

test_written = []
test_patient_order = {}

i = 0

for patient in test_patient_image_count.keys():
    mask_arr = np.ones((longest,))
    image_arr = np.zeros((longest, IMG_SIZE, IMG_SIZE, 3))
    
    test_masks.append(mask_arr)
    test_images.append(image_arr)
    
    test_written.append(0)
    test_patient_order[patient] = i
    i += 1

In [44]:
test_longest = max(test_patient_image_count.values())
print("Longest sequence is: {}".format(test_longest))

Longest sequence is: 240


In [45]:
print(test_patient_image_count.values())

dict_values([240, 46, 28, 38, 29, 55, 12, 16, 7, 12, 66, 90, 9, 29, 4, 93, 28, 29, 20, 28, 12, 29, 12, 24, 6, 108, 41, 80, 11, 11, 13, 50, 16, 21, 16, 17, 43, 49, 34, 63, 52, 38, 29, 22, 19, 9, 62, 27, 29, 21, 11, 31, 3, 16, 30, 5, 20, 15, 4, 10, 21, 10, 22, 68, 35, 8, 12, 58, 9, 58, 5, 47, 22, 13, 15, 3, 5, 25, 63, 32, 20, 31, 6, 32, 22, 48, 28, 20, 5, 5, 67, 21, 22, 26, 9, 12, 32, 8, 62, 43, 5, 29, 21, 21, 20, 47, 23, 9, 14, 20, 38, 33, 17, 15, 39, 33, 36, 10, 63, 39, 17, 37, 8, 4, 15, 6, 9, 37, 21, 20, 33, 10, 12, 52, 41, 16, 7, 10, 29, 40, 7, 63, 13, 10, 38, 30, 49, 16, 26, 14, 17, 28, 22, 44, 25, 28, 19, 26, 27, 21, 9, 29, 33, 15, 50, 8, 6, 5, 47, 15, 29, 13, 5, 7, 23, 21, 18, 6, 22, 30, 8, 14, 19, 20, 10, 11, 3, 13, 4, 22, 14, 22, 37, 46, 11, 21, 29, 10, 6, 3, 5, 26, 8, 12, 9, 46, 48, 22, 13, 7, 6, 30, 30, 14, 53, 13, 4, 24, 24, 29, 23, 4, 46, 18, 12, 10, 10, 15, 8, 8, 7, 40, 24, 26, 36, 51, 5, 9, 10, 8, 23, 11, 7, 19, 14, 16, 9, 14, 4, 4, 41, 10, 21, 17, 3, 9, 54, 12, 16, 52, 8,

In [ ]:
model.evaluate([val_images, val_masks], val_targets)

In [39]:
for p in test_patient_image_count.values():
    if p >= 115:
        print(p)

240


In [46]:
test_image_orders = {}
bad_images = []

In [47]:
for image in test_dataset:
    # Parse image.
    example = tf.io.parse_single_example(image, feature_description_test)
    img = tf.io.decode_image(example['image'], expand_animations = False, channels=channels)
    img = tf.cast(img, tf.float32)
    img = (img/127.5) - 1
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))

    # Get image name.
    name = example['image_name'].numpy().decode("utf_8")
    
    # Work out which patient this belongs to.
    patient = test_image_to_patient[name]
    
    # Find out patient's index.
    patient_index = test_patient_order[patient]
    
    if patient_index not in test_image_orders:
        test_image_orders[patient_index] = []
    
    test_image_orders[patient_index].append(name)
    
    # Find how many images have been written to this patient.
    written_index = test_written[patient_index]
    #if written_index >= 115:
    #    bad_images.append(name)
    #    continue
    
    # Write image.
    test_images[patient_index][written_index, :, :, :] = img
        
    # Set mask to 0.
    test_masks[patient_index][written_index] = 0
    
    # Increment written value.
    test_written[patient_index] += 1

In [48]:
len(bad_images)

0

In [49]:
test_images.shape

AttributeError: 'list' object has no attribute 'shape'

In [50]:
for i in range(len(test_masks)):
    test_masks[i] = np.reshape(test_masks[i], (1, 1, -1))

In [51]:
test_images = np.array(test_images)
test_masks = np.array(test_masks)

In [59]:
predictions = model.predict([test_images, test_masks])

In [ ]:
predictions[0]

In [60]:
import csv

patient_index = 0

with open("submission.csv", "w") as csvfile:
    writer = csv.writer(csvfile, delimiter=",")
    writer.writerow(["image_name", "target"])
    for patient in predictions:
        num_ims = min(len(test_image_orders[patient_index]), 240)
        for i in range(num_ims):
            image = patient[i]
            name = test_image_orders[patient_index][i]
            prounded = f"{image:.9f}"
            writer.writerow([name, prounded])
        patient_index += 1

In [ ]:
import random

with open("submission.csv", "a") as csvfile:
    writer = csv.writer(csvfile, delimiter=",")
    for b in bad_images:
        prounded = f"{random.random():.9f}"
        writer.writerow([b, prounded])